<a href="https://colab.research.google.com/github/Jawakar-7/Lesion-Detection-using-Mammograms/blob/main/Lesions_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! git clone https://github.com/Jawakar-7/Lesion-Detection-using-Mammograms

Cloning into 'Lesion-Detection-using-Mammograms'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
Receiving objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0


In [4]:
! mkdir ~/.kaggle
! cp /content/Lesion-Detection-using-Mammograms/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d skooch/ddsm-mammography

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 2.88G/2.88G [00:29<00:00, 182MB/s]
100% 2.88G/2.88G [00:29<00:00, 103MB/s]


In [5]:
import zipfile
path= '/content/ddsm-mammography.zip'  #  path to  zip file
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')  #extraction path
